## MNIST 

In [ ]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import pdb 

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
num_epochs=5
num_class=10
batch_size=100
learning_rate=0.001


In [ ]:
train_dataset=torchvision.datasets.MNIST(root='../\python_workshop\data/',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True                                         
                                         )
test_dataset=torchvision.datasets.MNIST(root='../\python_workshop\data/',
                                         train=False,
                                         transform=transforms.ToTensor()                                         
                                         )

# 새 섹션

In [ ]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                    batch_size=batch_size,
                                    shuffle=True
                                    )
test_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                    batch_size=batch_size,
                                    shuffle=False
                                    )

ConvNet class 뜯어보기
CLASStorch.nn.Conv2d(in_channels: int, out_channels: int, kernel_size: Union[T, Tuple[T, T]], stride: Union[T, Tuple[T, T]] = 1, padding: Union[T, Tuple[T, T]] = 0, dilation: Union[T, Tuple[T, T]] = 1, groups: int = 1, bias: bool = True, padding_mode: str = 'zeros')

'''
kernel = 필터
'''

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_class=10):
        super(ConvNet,self).__init__()
        self.layer1=nn.Sequential(
            # nn.Conv2d(입력 채널의 수,출력 채널 수,필터 사이즈=n,stride=n,padding=n),
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.ReLU())
        self.layer2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer3=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.ReLU())
        self.layer4 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.layer5=nn.Linear(7*7*32,num_class)
    
    def forward(self,x):
        # 2. Debugging
        # pdb.set_trace()
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=out.reshape(out.size(0),-1) #2차원을 펼쳐서 1차원으로 만들기
        out=self.layer5(out)
        return out


In [ ]:
model=ConvNet(num_class).to(device)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
total_step=len(train_loader)    # MNIST데이터의 경우 600번
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)
        
        # 1. Debugging
        # pdb.set_trace() # l,q... 중간중간 값 확인 가능
        preds=model(images)
         # 3. Debugging
        # pdb.set_trace()
        loss=loss_function(preds,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1)%100==0:
            print("Epoch [{}/{} | Step [{}/{}] | Loss:{:.3f}]"
            .format(epoch+1,num_epochs,i+1,total_step,loss.item()))


In [ ]:
model.eval()    # 모델을 eval()
with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)

        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    print("Test Acc : {}%".format(100*correct/total))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
for i in range(10):
    image=np.array(images[i,0].detach().cpu())
    label=labels[i].item()
    pred=predicted[i].item()
    
    plt.figure(figsize=(2,2))
    plt.title(f'label : {label}, Predicted : {pred}')
    plt.imshow(image*255)
